In [1]:
import torch
from argparse import  Namespace

## Phoneme Recognition Model

In [2]:
from phoneme_GAT.phoneme_model import BaseModule, load_phoneme_model, optim_param

/home/ay/anaconda3/envs/phoneme_deepfake_detection/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from ay2.tools.text._phonemes import Phonemer_Tokenizer_Recombination

In [4]:
network_param = Namespace(
    network_name="WavLM",
    pretrained_path = "/home/ay/data/phonemes/wavlm/best-epoch=42-val-per=0.407000.ckpt",
    freeze=True,
    freeze_transformer=True,
    eos_token="</s>",
    bos_token="<s>",
    unk_token="<unk>",
    pad_token="<pad>",
    word_delimiter_token="|",
    vocab_size=200,
)

To build the phoneme recognition model,
1. you must specify the pretrained_path!!!! Please download the provided pretrained phoneme model; or you can train yourself model through `train_phoneme_model.py`.
2. in the `load_phoneme_model` function, you have to change the correct `vocab_path`

In [5]:
total_num_phonemes = 687  ## 198, or 687

phoneme_model = load_phoneme_model(
    network_name=network_param.network_name,
    pretrained_path=network_param.pretrained_path,
    total_num_phonemes=total_num_phonemes,
)
assert len(phoneme_model.tokenizer.total_phonemes) == total_num_phonemes

Now, load vocab json files from  /home/ay/tmp/PLFD-ADD/vocab_phoneme Please make sure the vocab files are correct


/home/ay/anaconda3/envs/phoneme_deepfake_detection/lib/python3.9/site-packages/lightning_fabric/utilities/cloud_io.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retur

Load WavLM model!!!!!!!


Some weights of WavLMForCTC were not initialized from the model checkpoint at /home/ay/.cache/huggingface/hub/models--microsoft--wavlm-base/snapshots/efa81aae7ff777e464159e0f877d54eac5b84f81/ and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([687, 768])


# Load model

## Audio model

In [8]:
from phoneme_GAT.modules import Phoneme_GAT_lit,Phoneme_GAT

In [9]:
audio_model = Phoneme_GAT(
    backbone='wavlm',
    use_raw=0,
    use_GAT=1,
    n_edges=10,
)

Now, load vocab json files from  /home/ay/tmp/PLFD-ADD/vocab_phoneme Please make sure the vocab files are correct


/home/ay/anaconda3/envs/phoneme_deepfake_detection/lib/python3.9/site-packages/lightning_fabric/utilities/cloud_io.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retur

Load WavLM model!!!!!!!


Some weights of WavLMForCTC were not initialized from the model checkpoint at /home/ay/.cache/huggingface/hub/models--microsoft--wavlm-base/snapshots/efa81aae7ff777e464159e0f877d54eac5b84f81/ and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([687, 768])


In [11]:
x = torch.randn(3, 1, 48000)
num_frames = torch.full((x.shape[0],), 48000 // 320 - 1) # (batch_size,)
res = audio_model(x, num_frames=num_frames)

In [12]:
for key, value in res.items():
    print(key, value.shape)

logit torch.Size([3])
hidden_states torch.Size([3, 768])
phoneme_feat torch.Size([3, 149, 768])
encoder_feat torch.Size([3, 149, 768])
phoneme_cls_logit torch.Size([3, 687])
phoneme_cls_label torch.Size([3])
aug_logit torch.Size([3])
aug_frame_logit torch.Size([3])
aug_labels torch.Size([3])


In [13]:
from argparse import Namespace

# Construct the configuration using Namespace
cfg = Namespace(
    PhonemeGAT=Namespace(
        backbone="wavlm",  # wavlm or wav2vec
        use_raw=False,              # whether to use raw transformer as the backbone
        use_GAT=True,              # whether to use GAT
        n_edges=10,                # the nubmer of edges for each node in the GAT
        use_aug=True,              # whether to use data augmentation in the training
        use_pool=True,            # whether to use pooling
        use_clip=True,             # whether to use clip loss
    )
)

## Lit model

We use the pytorch Lightning module to train the model, where we define the train step, validation/predict step, loss function and optimizer.

In [14]:
audio_model_lit = Phoneme_GAT_lit(cfg=cfg)

Now, load vocab json files from  /home/ay/tmp/PLFD-ADD/vocab_phoneme Please make sure the vocab files are correct


/home/ay/anaconda3/envs/phoneme_deepfake_detection/lib/python3.9/site-packages/lightning_fabric/utilities/cloud_io.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retur

Load WavLM model!!!!!!!


Some weights of WavLMForCTC were not initialized from the model checkpoint at /home/ay/.cache/huggingface/hub/models--microsoft--wavlm-base/snapshots/efa81aae7ff777e464159e0f877d54eac5b84f81/ and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([687, 768])
